In [1]:

import os
import time
import csv
import numpy as np
from PIL import Image
from decimal import Decimal, getcontext
from collections import defaultdict

# Set high precision for Decimal arithmetic
getcontext().prec = 200
BLOCK_SIZE = 32

# CALIC prediction
def calic_predict(img_arr):
    height, width = img_arr.shape
    pred_arr = np.zeros_like(img_arr, dtype=np.int16)
    for i in range(height):
        for j in range(width):
            if i == 0 and j == 0:
                pred = 0
            elif i == 0:
                pred = img_arr[i, j-1]
            elif j == 0:
                pred = img_arr[i-1, j]
            else:
                pred = (img_arr[i-1, j] + img_arr[i, j-1]) // 2
            pred_arr[i, j] = int(img_arr[i, j]) - int(pred)
    return pred_arr

# CALIC reconstruction
def calic_reconstruct(pred_arr):
    height, width = pred_arr.shape
    recon_arr = np.zeros_like(pred_arr, dtype=np.uint8)
    for i in range(height):
        for j in range(width):
            if i == 0 and j == 0:
                pred = 0
            elif i == 0:
                pred = recon_arr[i, j-1]
            elif j == 0:
                pred = recon_arr[i-1, j]
            else:
                pred = (recon_arr[i-1, j] + recon_arr[i, j-1]) // 2
            recon_arr[i, j] = (int(pred) + int(pred_arr[i, j])) % 256
    return recon_arr

# Arithmetic Encoder
class ArithmeticEncoder:
    def __init__(self, freq):
        self.freq = freq
        self.total = sum(freq.values())
        self.cum_freq = self.build_cum_freq()
        self.low = Decimal(0)
        self.high = Decimal(1)

    def build_cum_freq(self):
        cum_freq = {}
        total = 0
        for symbol in range(256):
            cum_freq[symbol] = total
            total += self.freq.get(symbol, 0)
        return cum_freq

    def encode(self, data):
        self.low = Decimal(0)
        self.high = Decimal(1)
        for symbol in data:
            range_ = self.high - self.low
            low_count = self.cum_freq[symbol]
            high_count = low_count + self.freq[symbol]
            self.high = self.low + range_ * Decimal(high_count) / self.total
            self.low = self.low + range_ * Decimal(low_count) / self.total
        return (self.low + self.high) / 2

# Arithmetic Decoder
class ArithmeticDecoder:
    def __init__(self, freq, encoded_value, data_length):
        self.low = Decimal(0)
        self.high = Decimal(1)
        self.value = encoded_value
        self.freq = freq
        self.total = sum(freq.values())
        self.cum_freq = self.build_cum_freq()
        self.symbols = list(range(256))
        self.data_length = data_length

    def build_cum_freq(self):
        cum_freq = {}
        total = 0
        for symbol in range(256):
            cum_freq[symbol] = total
            total += self.freq.get(symbol, 0)
        return cum_freq

    def decode(self, length):
        decoded = []
        for _ in range(length):
            range_ = self.high - self.low
            if range_ == 0:
                decoded.append(max(self.freq.items(), key=lambda x: x[1])[0])
                continue

            scaled_value = (self.value - self.low) * self.total / range_
            for symbol in self.symbols:
                low_count = self.cum_freq[symbol]
                high_count = low_count + self.freq[symbol]
                if low_count <= scaled_value < high_count:
                    decoded.append(symbol)
                    self.high = self.low + range_ * Decimal(high_count) / self.total
                    self.low = self.low + range_ * Decimal(low_count) / self.total
                    break
        return decoded



In [ ]:
# MAIN PROCESSING LOOP

input_folder = "org_ecg_10sec_resize"
output_folder = "after compress calic arithmetic"
os.makedirs(output_folder, exist_ok=True)

durations = []

for filename in os.listdir(input_folder):
    if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp")):
        start_time = time.time()

        # Load image
        image_path = os.path.join(input_folder, filename)
        img = Image.open(image_path).convert('L')
        arr = np.array(img, dtype=np.uint8)

        # CALIC prediction
        pred_arr = calic_predict(arr)
        shifted = (pred_arr + 255) % 256
        flat = shifted.flatten().tolist()
        shape = arr.shape
        total_pixels = shape[0] * shape[1]

        # Build frequency table
        freq = defaultdict(int, {i: 1 for i in range(256)})
        for val in flat:
            freq[val] += 1

        # Arithmetic encoding
        encoder = ArithmeticEncoder(freq)
        blocks = [flat[i:i+BLOCK_SIZE] for i in range(0, len(flat), BLOCK_SIZE)]
        encoded_blocks = [str(encoder.encode(block)) for block in blocks]
        block_sizes = [len(block) for block in blocks]

        # Arithmetic decoding
        decoded_flat = []
        for encoded_val_str, block_len in zip(encoded_blocks, block_sizes):
            encoded_val = Decimal(encoded_val_str)
            decoder = ArithmeticDecoder(freq, encoded_val, block_len)
            decoded_flat.extend(decoder.decode(block_len))

        # Shift back and reconstruct
        shifted_back = (np.array(decoded_flat, dtype=np.int16).reshape(shape) - 255) % 256
        recon_img = calic_reconstruct(shifted_back)

        # Save reconstructed image
        output_path = os.path.join(output_folder, filename)
        Image.fromarray(recon_img).save(output_path)

        end_time = time.time()
        duration = end_time - start_time
        durations.append((filename, duration))
        print(f"{filename}: processed in {duration:.4f} seconds")

# Save durations to CSV
with open("calic_arithmetic.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Image Name", "Duration (seconds)"])
    writer.writerows(durations)


C:\Users\Jiawei\AppData\Local\Temp\ipykernel_25148\529002232.py:26: RuntimeWarning: overflow encountered in scalar add
  pred = (img_arr[i-1, j] + img_arr[i, j-1]) // 2
C:\Users\Jiawei\AppData\Local\Temp\ipykernel_25148\529002232.py:43: RuntimeWarning: overflow encountered in scalar add
  pred = (recon_arr[i-1, j] + recon_arr[i, j-1]) // 2


ecg_record_100.png: processed in 11.5657 seconds
ecg_record_101.png: processed in 11.2338 seconds
ecg_record_102.png: processed in 11.4532 seconds
ecg_record_103.png: processed in 11.3392 seconds
ecg_record_104.png: processed in 11.3007 seconds
ecg_record_105.png: processed in 11.3719 seconds
ecg_record_106.png: processed in 11.4831 seconds
ecg_record_107.png: processed in 11.2035 seconds
ecg_record_108.png: processed in 11.1726 seconds
ecg_record_109.png: processed in 11.3825 seconds
ecg_record_111.png: processed in 11.4693 seconds
ecg_record_112.png: processed in 11.7858 seconds
ecg_record_113.png: processed in 11.7735 seconds
ecg_record_114.png: processed in 11.3034 seconds
ecg_record_115.png: processed in 11.3393 seconds
ecg_record_116.png: processed in 11.1378 seconds
ecg_record_117.png: processed in 11.4010 seconds
ecg_record_118.png: processed in 11.2923 seconds
ecg_record_119.png: processed in 11.2942 seconds
ecg_record_121.png: processed in 11.1303 seconds
ecg_record_122.png: 